In [52]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType


In [53]:
# Prepare data
start = '2019-01-01 UTC'  # crypto is in UTC
end = datetime.utcnow()
dir_name = '/Users/andre/Documents/Python/trading_clone_singapore_DO/trading_singapore_digitalocean/vectorBT'
os.makedirs(dir_name, exist_ok=True)

# Function to download and save data
def download_and_save_data(symbol, file_name, overwrite=False):
    joined = os.path.join(dir_name, file_name)
    if os.path.exists(joined) and not overwrite:
        data = pd.read_pickle(joined)
        print(f'File {file_name} exists already')
    else:
        data = vbt.CCXTData.download(exchange='kucoin', symbols=symbol, timeframe='1d', start=start, end=end).get()
        data.to_pickle(joined)
        print(f'Downloaded and saved {symbol} data')
    return data

# List of assets
assets = ['BTC']
divider = '/'  # Divider for the assets
base_value = '/USDT'  # Base value for the assets

# Dictionary to store data
data_dict = {}

# Download data for each asset
for asset in assets:
    symbol = asset.lower()
    file_name = f'{symbol}_price_1d.pkl'
    data_dict[symbol] = download_and_save_data(asset + base_value, file_name, overwrite=False)

# Align the DataFrames by their index
aligned_data = [data_dict[symbol].align(data_dict[assets[0].lower()], join='inner')[0] for symbol in data_dict]

# Concatenate the aligned DataFrames
comb_price = pd.concat(aligned_data, axis=1, keys=[asset for asset in assets])
comb_price.columns.names = ['symbol', 'attribute']

# Create DataFrames for Open, High, Low, and Close prices
comb_open = pd.concat([data_dict[symbol]['Open'] for symbol in data_dict], axis=1, keys=[f'{symbol}_open' for symbol in data_dict])
comb_high = pd.concat([data_dict[symbol]['High'] for symbol in data_dict], axis=1, keys=[f'{symbol}_high' for symbol in data_dict])
comb_low = pd.concat([data_dict[symbol]['Low'] for symbol in data_dict], axis=1, keys=[f'{symbol}_low' for symbol in data_dict])
comb_close = pd.concat([data_dict[symbol]['Close'] for symbol in data_dict], axis=1, keys=[f'{symbol}_close' for symbol in data_dict])

# Display the resulting DataFrames
print("Open Prices:")
print(comb_open.head())

print("\nHigh Prices:")
print(comb_high.head())

print("\nLow Prices:")
print(comb_low.head())

print("\nClose Prices:")
print(comb_close.head())


File btc_price_1d.pkl exists already
Open Prices:
                              btc_open
Open time                             
2019-01-01 00:00:00+00:00  3700.170853
2019-01-02 00:00:00+00:00  3799.644687
2019-01-03 00:00:00+00:00  3857.633097
2019-01-04 00:00:00+00:00  3767.854300
2019-01-05 00:00:00+00:00  3789.000000

High Prices:
                              btc_high
Open time                             
2019-01-01 00:00:00+00:00  3805.841739
2019-01-02 00:00:00+00:00  3880.257733
2019-01-03 00:00:00+00:00  3862.600830
2019-01-04 00:00:00+00:00  3821.195343
2019-01-05 00:00:00+00:00  3838.674540

Low Prices:
                               btc_low
Open time                             
2019-01-01 00:00:00+00:00  3645.525763
2019-01-02 00:00:00+00:00  3750.000001
2019-01-03 00:00:00+00:00  3728.917225
2019-01-04 00:00:00+00:00  3706.000000
2019-01-05 00:00:00+00:00  3756.410862

Close Prices:
                             btc_close
Open time                             
2019-01-01 

# creating indicator to set signals long/short 

In [54]:
def ma_cross_rsi(close_price, ma1_fast, ma2_slow, rsi_buy_threshold, rsi_sell_threshold, ma_diff_threshold=0.05):
    # Calculate moving averages
    ma1 = vbt.MA.run(close_price, window=ma1_fast).ma.to_numpy()
    ma2 = vbt.MA.run(close_price, window=ma2_slow).ma.to_numpy()
    
    # Calculate RSI and determine buy/sell signals
    rsi = vbt.RSI.run(close_price, window=14).rsi.to_numpy()
    rsi_buy = rsi > rsi_buy_threshold
    rsi_sell = rsi < rsi_sell_threshold
    
    # Generate buy and sell signals
    signal_buy = (close_price > ma1) & rsi_buy & (ma1 > ma2)
    signal_sell = (close_price < ma1) & rsi_sell & (ma1 < ma2)

    caution_long = (close_price < ma1) & (ma1 > ma2)
    caution_short = (close_price > ma1) & (ma1 < ma2)
    
    # Check if ma1 and ma2 are too close to each other (percentage difference)
    percentage_diff = np.abs(ma1 - ma2) / ma2
    undecided = percentage_diff < ma_diff_threshold
    
    # Define conditions and corresponding values for signals
    conditions = [
        signal_buy & ~undecided,
        caution_long & ~undecided,
        signal_sell & ~undecided,
        caution_short & ~undecided,
        undecided
    ]

    values = [
        100,  # Strong buy signal (close price above short trend MA and RSI above the buy threshold)
        50,   # Caution long
        -100, # Strong sell signal (close price below short trend MA and RSI below the sell threshold)
        -50,  # Caution short
        0     # Undecided
    ]

    # Use np.select to apply the conditions and assign the corresponding values
    signal = np.select(conditions, values, default=0)  # 0: No signal (default value when none of the conditions are met)

    return signal

# Create the indicator factory
ma_cross_slow = vbt.IndicatorFactory(
    class_name='MA_Cross_rsi',
    short_name='ma_c_slow',
    input_names=['Close'],
    param_names=['ma1_fast', 'ma2_slow', 'rsi_buy_threshold', 'rsi_sell_threshold'],
    output_names=['ma_cross_rsi']
).from_apply_func(ma_cross_rsi, ma1_fast=20, ma2_slow=50, rsi_buy_threshold=70, rsi_sell_threshold=30)

# Define window sizes and RSI thresholds using np.arange
ma_trend_fast = np.arange(20, 51, 5)  # Range from 20 to 50 with step 5
ma_trend_slow = np.arange(50, 101, 10)  # Range from 50 to 100 with step 10
rsi_buy_threshold = np.arange(60, 81, 5)  # Range from 60 to 80 with step 5
rsi_sell_threshold = np.arange(25, 36, 5)  # Range from 25 to 35 with step 5

# # Define window sizes and RSI thresholds
# ma_trend_fast = 50
# ma_trend_slow = 100
# rsi_buy_threshold = 70
# rsi_sell_threshold = 30

# Run the indicator
trend = ma_cross_slow.run(
    comb_close,
    ma1_fast=ma_trend_fast,
    ma2_slow=ma_trend_slow,
    rsi_buy_threshold=rsi_buy_threshold,
    rsi_sell_threshold=rsi_sell_threshold,
    param_product=True
)

trend.ma_cross_rsi.columns[0]

MultiIndex([(20,  50, 60, 25),
            (20,  50, 60, 30),
            (20,  50, 60, 35),
            (20,  50, 65, 25),
            (20,  50, 65, 30),
            (20,  50, 65, 35),
            (20,  50, 70, 25),
            (20,  50, 70, 30),
            (20,  50, 70, 35),
            (20,  50, 75, 25),
            ...
            (50, 100, 65, 35),
            (50, 100, 70, 25),
            (50, 100, 70, 30),
            (50, 100, 70, 35),
            (50, 100, 75, 25),
            (50, 100, 75, 30),
            (50, 100, 75, 35),
            (50, 100, 80, 25),
            (50, 100, 80, 30),
            (50, 100, 80, 35)],
           names=['ma_c_slow_ma1_fast', 'ma_c_slow_ma2_slow', 'ma_c_slow_rsi_buy_threshold', 'ma_c_slow_rsi_sell_threshold'], length=630)

# caluculation for needed input of columns to match indicator possibilities
- check how many assets will be tested 
- check how many columns (different setups) will be produces
- devide by aamount of asset to to find the number of copy needed for specific values closing price
- need to be done because from_order funct needs goes through column by column to to calculate the trades 
- that means every indicator entry column need a asset close column to check against

In [64]:
print(len(trend.ma_cross_rsi.columns)) 
print(len(assets))
needed_copies = int(len(trend.ma_cross_rsi.columns)/len(assets))
print(needed_copies)

630
1
630


# copy dataframe that contains assest closing price 

In [65]:
n = int(needed_copies)

# Create DataFrames for Open, High, Low, and Close prices with n copies
comb_open = pd.concat([data_dict[symbol]['Open'].rename(f'{symbol}_open{i+1}') for i in range(n) for symbol in data_dict], axis=1)
comb_high = pd.concat([data_dict[symbol]['High'].rename(f'{symbol}_high{i+1}') for i in range(n) for symbol in data_dict], axis=1)
comb_low = pd.concat([data_dict[symbol]['Low'].rename(f'{symbol}_low{i+1}') for i in range(n) for symbol in data_dict], axis=1)
comb_close = pd.concat([data_dict[symbol]['Close'].rename(f'{symbol}_close{i+1}') for i in range(n) for symbol in data_dict], axis=1)

# Display the resulting DataFrames with n copies
print("Open Prices with n copies:")
print(comb_open.head())

print("\nHigh Prices with n copies:")
print(comb_high.head())

print("\nLow Prices with n copies:")
print(comb_low.head())

print("\nClose Prices with n copies:")
print(comb_close.head())

Open Prices with n copies:
                             btc_open1    btc_open2    btc_open3    btc_open4  \
Open time                                                                       
2019-01-01 00:00:00+00:00  3700.170853  3700.170853  3700.170853  3700.170853   
2019-01-02 00:00:00+00:00  3799.644687  3799.644687  3799.644687  3799.644687   
2019-01-03 00:00:00+00:00  3857.633097  3857.633097  3857.633097  3857.633097   
2019-01-04 00:00:00+00:00  3767.854300  3767.854300  3767.854300  3767.854300   
2019-01-05 00:00:00+00:00  3789.000000  3789.000000  3789.000000  3789.000000   

                             btc_open5    btc_open6    btc_open7    btc_open8  \
Open time                                                                       
2019-01-01 00:00:00+00:00  3700.170853  3700.170853  3700.170853  3700.170853   
2019-01-02 00:00:00+00:00  3799.644687  3799.644687  3799.644687  3799.644687   
2019-01-03 00:00:00+00:00  3857.633097  3857.633097  3857.633097  3857.633097   


In [66]:
import pandas as pd

comb_close_copy = comb_close.copy()
df_C_price = comb_close_copy
df_indi = trend.ma_cross_rsi


# Assuming df_indi and df_C_price are your DataFrames
# Check if they have the same shape
if df_indi.shape == df_C_price.shape:
    # Rename the columns of df_C_price to match the column names of df_indi
    df_C_price.columns = df_indi.columns
    print("Columns of df_C_price have been renamed to match df_indi.")
else:
    print("The DataFrames do not have the same shape.")

# Display the resulting DataFrame
df_C_price.head()

Columns of df_C_price have been renamed to match df_indi.


ma_c_slow_ma1_fast                     20                            \
ma_c_slow_ma2_slow                    50                              
ma_c_slow_rsi_buy_threshold            60                             
ma_c_slow_rsi_sell_threshold           25           30           35   
Open time                                                             
2019-01-01 00:00:00+00:00     3799.644687  3799.644687  3799.644687   
2019-01-02 00:00:00+00:00     3856.504318  3856.504318  3856.504318   
2019-01-03 00:00:00+00:00     3765.999334  3765.999334  3765.999334   
2019-01-04 00:00:00+00:00     3791.461260  3791.461260  3791.461260   
2019-01-05 00:00:00+00:00     3772.156039  3772.156039  3772.156039   

ma_c_slow_ma1_fast                                                   \
ma_c_slow_ma2_slow                                                    
ma_c_slow_rsi_buy_threshold            65                             
ma_c_slow_rsi_sell_threshold           25           30           35   
Open time                                                             
2019-01-01 00:00:00+00:00     3799.644687  3799.644687  3799.644687   
2019-01-02 00:00:00+00:00     3856.504318  3856.504318  3856.504318   
2019-01-03 00:00:00+00:00     3765.999334  3765.999334  3765.999334   
2019-01-04 00:00:00+00:00     3791.461260  3791.461260  3791.461260   
2019-01-05 00:00:00+00:00     3772.156039  3772.156039  3772.156039   

ma_c_slow_ma1_fast                                                   \
ma_c_slow_ma2_slow                                                    
ma_c_slow_rsi_buy_threshold            70                             
ma_c_slow_rsi_sell_threshold           25           30           35   
Open time                                                             
2019-01-01 00:00:00+00:00     3799.644687  3799.644687  3799.644687   
2019-01-02 00:00:00+00:00     3856.504318  3856.504318  3856.504318   
2019-01-03 00:00:00+00:00     3765.999334  3765.999334  3765.999334   
2019-01-04 00:00:00+00:00     3791.461260  3791.461260  3791.461260   
2019-01-05 00:00:00+00:00     3772.156039  3772.156039  3772.156039   

ma_c_slow_ma1_fast                         ...           50               \
ma_c_slow_ma2_slow                         ...          100                
ma_c_slow_rsi_buy_threshold            75  ...           65           70   
ma_c_slow_rsi_sell_threshold           25  ...           35           25   
Open time                                  ...                             
2019-01-01 00:00:00+00:00     3799.644687  ...  3799.644687  3799.644687   
2019-01-02 00:00:00+00:00     3856.504318  ...  3856.504318  3856.504318   
2019-01-03 00:00:00+00:00     3765.999334  ...  3765.999334  3765.999334   
2019-01-04 00:00:00+00:00     3791.461260  ...  3791.461260  3791.461260   
2019-01-05 00:00:00+00:00     3772.156039  ...  3772.156039  3772.156039   

ma_c_slow_ma1_fast                                                   \
ma_c_slow_ma2_slow                                                    
ma_c_slow_rsi_buy_threshold                                      75   
ma_c_slow_rsi_sell_threshold           30           35           25   
Open time                                                             
2019-01-01 00:00:00+00:00     3799.644687  3799.644687  3799.644687   
2019-01-02 00:00:00+00:00     3856.504318  3856.504318  3856.504318   
2019-01-03 00:00:00+00:00     3765.999334  3765.999334  3765.999334   
2019-01-04 00:00:00+00:00     3791.461260  3791.461260  3791.461260   
2019-01-05 00:00:00+00:00     3772.156039  3772.156039  3772.156039   

ma_c_slow_ma1_fast                                                   \
ma_c_slow_ma2_slow                                                    
ma_c_slow_rsi_buy_threshold                                      80   
ma_c_slow_rsi_sell_threshold           30           35           25   
Open time                                                             
2019-01-01 00:00:00+00:00     3799.64468

In [67]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices):
    close_price = c.close[c.i, c.col]
    # low_price = low[c.i, c.col]
    # high_price = high[c.i, c.col]
    # open_price = open_[c.i, c.col]    
    # print("INDEX", c.i)
    # print("COL", c.col)
    # print('close price:', close_price)
    # print('high price:', high_price)
    # print('low price:', low_price)
    # print('open price:', open_price)
    # print('position size :', c.position_now)
    # print('cash:', c.cash_now)
    # print('entries:', entries[c.i, c.col])
    # print()

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        if low[c.i-1,c.col] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=sl_prices[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly)

            # print('sl hit at Index:', c.i-1)
            # print('sl price:', sl_prices[c.i])
            return value
        
        # check condition to adjust sl 
        # find lowest low in the last 5 bars
        # set sl n% lower than the lowest low
        # if new sl is higher than the current sl update
        # if new sl is lower than the current sl do nothing

        # sl update for long position
        if c.i >= 5:  # Ensure there are enough bars to look back
            lowest_low = np.min(low[c.i-2:c.i,c.col])  # Find the lowest low in the last 5 bars
            if (entries[c.i, c.col] == 75):
                # print('sl update before',c.i,sl_prices[c.i])
                update1 = lowest_low * 0.90
                if update1 > sl_prices[c.i]:
                    sl_prices[:]= update1
                    # print('sl update after at index',c.i,sl_prices[c.i])
            if (entries[c.i, c.col] == 50):  
                update2 = lowest_low * 0.95
                if update2 > sl_prices[c.i]:
                    # print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= update2          
                # print('sl update after at index',c.i,sl_prices[c.i])
            if (entries[c.i, c.col] == 25):
                update3 = lowest_low * 0.98
                if update3 > sl_prices[c.i]:
                    # print('sl update before',c.i,sl_prices[c.i])
                    update3 = lowest_low * 0.98
                # print('sl update after at index',c.i,sl_prices[c.i])

    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i, c.col] == 100:
            sl_price = close_price * 0.90
            order = vbt.portfolio.nb.order_nb(
                size=0.1,  # Adjusted order size
                price=close_price,  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.0,  # No fees
                slippage=0.0,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )
            sl_prices[:] = sl_price 
            # print('sl to put in array price:', sl_price)
            # print('sl array',sl_prices[1])   # Save the SL price using a single index
            # print('long order', c.i,order)
            # print()
            return order
    


    return vbt.portfolio.enums.NoOrder


open_ = comb_open.to_numpy()
high = comb_high.to_numpy()
low = comb_low.to_numpy()
close = df_C_price
entries = trend.ma_cross_rsi.to_numpy()

# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,  # Pass the SL prices array
    init_cash=100000,  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable

# Optional: Plot equity curve
# pf.plot().show()



Total Return: ma_c_slow_ma1_fast  ma_c_slow_ma2_slow  ma_c_slow_rsi_buy_threshold  ma_c_slow_rsi_sell_threshold
20                  50                  60                           25                              0.341010
                                                                     30                              0.341010
                                                                     35                              0.341010
                                        65                           25                              0.337347
                                                                     30                              0.337347
                                                                                                       ...   
50                  100                 75                           30                              0.456953
                                                                     35                              0.456953
        

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(20, 50, 60, 25)",2019-03-10 00:00:00+00:00,2.553887,3915.600,0.0,Buy
1,1,"(20, 50, 60, 25)",2019-06-05 00:00:00+00:00,2.553887,8023.415,0.0,Sell
2,2,"(20, 50, 60, 25)",2019-06-17 00:00:00+00:00,1.186773,9310.200,0.0,Buy
3,3,"(20, 50, 60, 25)",2019-07-14 00:00:00+00:00,1.186773,10993.590,0.0,Sell
4,4,"(20, 50, 60, 25)",2020-01-18 00:00:00+00:00,1.261607,8916.300,0.0,Buy
...,...,...,...,...,...,...,...
15814,15814,"(50, 100, 80, 35)",2023-11-10 00:00:00+00:00,0.323046,37302.900,0.0,Buy
15815,15815,"(50, 100, 80, 35)",2024-01-23 00:00:00+00:00,0.323046,39932.870,0.0,Sell
15816,15816,"(50, 100, 80, 35)",2024-02-08 00:00:00+00:00,0.267959,45288.600,0.0,Buy
15817,15817,"(50, 100, 80, 35)",2024-04-14 00:00:00+00:00,0.267959,61692.810,0.0,Sell


In [68]:
pf.total_return().sort_values(ascending=False).head(20)

ma_c_slow_ma1_fast  ma_c_slow_ma2_slow  ma_c_slow_rsi_buy_threshold  ma_c_slow_rsi_sell_threshold
50                  100                 75                           35                              0.456953
                                                                     30                              0.456953
                                                                     25                              0.456953
                                        70                           35                              0.449354
                                                                     30                              0.449354
                                                                     25                              0.449354
                                        65                           35                              0.430483
                                                                     30                              0.430483
                      

: 

In [62]:
check = pf.orders.records_readable.sort_values(by='Timestamp')
check
pf.orders.records_readable.to_pickle('orders.pkl')
# print total return
check = pf.total_return().sort_values(ascending=False).head(5)
check.vbt.heatmap(x_level=entries, y_level='param_group',slider_level='symbol')


ValueError: 'symbol' is not in list

In [33]:
pf.total_return().max()
pf.total_return().idxmax()

'btc_close3242'